# First: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

Refer to ER Diagram - Fetch Analytics Engineer - Ipsita Mohapatra.pdf

# Second: Write queries that directly answer predetermined questions from a business stakeholder

### 1) What are the top 5 brands by receipts scanned for most recent month?

In [ ]:
WITH Recent_Month_Receipts AS (
    SELECT
        ri.brand_id,
        COUNT(DISTINCT r.receipt_id) AS receipt_count
    FROM
        Receipts r
    JOIN
        Receipt_Items ri ON r.receipt_id = ri.receipt_id
    WHERE
        r.date_scanned >= DATE_TRUNC('month', CURRENT_DATE) - INTERVAL '1 month'
        AND r.date_scanned < DATE_TRUNC('month', CURRENT_DATE)
    GROUP BY
        ri.brand_id
)
SELECT
    b.brand_name,
    rm.receipt_count
FROM
    Recent_Month_Receipts rm
JOIN
    Brands b ON rm.brand_id = b.brand_id
ORDER BY
    rm.receipt_count DESC
LIMIT 5;

### 2) How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [ ]:
WITH Recent_Month_Receipts AS (
    SELECT
        ri.brand_id,
        COUNT(DISTINCT r.receipt_id) AS receipt_count,
        'Recent Month' AS period
    FROM
        Receipts r
    JOIN
        Receipt_Items ri ON r.receipt_id = ri.receipt_id
    WHERE
        r.date_scanned >= DATE_TRUNC('month', CURRENT_DATE) - INTERVAL '1 month'
        AND r.date_scanned < DATE_TRUNC('month', CURRENT_DATE)
    GROUP BY
        ri.brand_id
),
Previous_Month_Receipts AS (
    SELECT
        ri.brand_id,
        COUNT(DISTINCT r.receipt_id) AS receipt_count,
        'Previous Month' AS period
    FROM
        Receipts r
    JOIN
        Receipt_Items ri ON r.receipt_id = ri.receipt_id
    WHERE
        r.date_scanned >= DATE_TRUNC('month', CURRENT_DATE) - INTERVAL '2 months'
        AND r.date_scanned < DATE_TRUNC('month', CURRENT_DATE) - INTERVAL '1 month'
    GROUP BY
        ri.brand_id
),
Ranked_Receipts AS (
    SELECT brand_id, receipt_count, period,
           RANK() OVER (PARTITION BY period ORDER BY receipt_count DESC) AS rank
    FROM (
        SELECT * FROM Recent_Month_Receipts
        UNION ALL
        SELECT * FROM Previous_Month_Receipts
    ) AS combined_receipts
)
SELECT
    b.brand_name,
    rr.period,
    rr.rank,
    rr.receipt_count
FROM
    Ranked_Receipts rr
JOIN
    Brands b ON rr.brand_id = b.brand_id
WHERE
    rr.rank <= 5
ORDER BY
    rr.period, rr.rank;


### 3) When considering average spend from receipts with 'rewardsReceiptStatus' of 'Accepted' or 'Rejected', which is greater?
In receipts.json, rewardsReceiptStatus is either FINISHED or REJECTED

In [ ]:
WITH Receipt_Status_Spend AS (
    SELECT
        r.rewards_receipt_status,
        AVG(r.total_spent) AS avg_spend
    FROM
        Receipts r
    WHERE
        r.rewards_receipt_status IN ('FINISHED', 'REJECTED')
    GROUP BY
        r.rewards_receipt_status
)
SELECT
    rewards_receipt_status,
    avg_spend
FROM
    Receipt_Status_Spend
ORDER BY
    avg_spend DESC;

### 4) When considering the total number of items purchased from receipts with 'rewardsReceiptStatus’ of 'Accepted' or 'Rejected', which is greater?
In receipts.json, rewardsReceiptStatus is either FINISHED or REJECTED

In [ ]:
WITH Receipt_Status_Items AS (
    SELECT
        r.rewards_receipt_status,
        SUM(ri.quantity_purchased) AS total_items
    FROM
        Receipts r
    JOIN
        Receipt_Items ri ON r.receipt_id = ri.receipt_id
    WHERE
        r.rewards_receipt_status IN ('FINISHED', 'REJECTED')
    GROUP BY
        r.rewards_receipt_status
)
SELECT
    rewards_receipt_status,
    total_items
FROM
    Receipt_Status_Items
ORDER BY
    total_items DESC;


### 5) Which brand has the most spend among users who were created within the past 6 months?

WITH Recent_Users AS (
    SELECT
        user_id
    FROM
        Users
    WHERE
        created_date >= CURRENT_DATE - INTERVAL '6 months'
),
Recent_User_Spend AS (
    SELECT
        ri.brand_id,
        SUM(r.total_spent) AS total_spent
    FROM
        Receipts r
    JOIN
        Receipt_Items ri ON r.receipt_id = ri.receipt_id
    JOIN
        Recent_Users u ON r.user_id = u.user_id
    GROUP BY
        ri.brand_id
)
SELECT
    b.brand_name,
    rus.total_spent
FROM
    Recent_User_Spend rus
JOIN
    Brands b ON rus.brand_id = b.brand_id
ORDER BY
    rus.total_spend DESC
LIMIT 1;


### 6) Which brand has the most transactions among users who were created within the past 6 months?

In [ ]:
WITH Recent_Users AS (
    SELECT
        user_id
    FROM
        Users
    WHERE
        created_date >= CURRENT_DATE - INTERVAL '6 months'
),
Recent_User_Transactions AS (
    SELECT
        ri.brand_id,
        COUNT(DISTINCT r.receipt_id) AS total_transactions
    FROM
        Receipts r
    JOIN
        Receipt_Items ri ON r.receipt_id = ri.receipt_id
    JOIN
        Recent_Users u ON r.user_id = u.user_id
    GROUP BY
        ri.brand_id
)
SELECT
    b.brand_name,
    rut.total_transactions
FROM
    Recent_User_Transactions rut
JOIN
    Brands b ON rut.brand_id = b.brand_id
ORDER BY
    rut.total_transactions DESC
LIMIT 1;


# Third: Evaluate Data Quality Issues

### Import & Clean Data

#### Basic Cleaning steps: 
- Extract data that is in nested dictionary
- Create separate table for Receipt_Items
- Create separate table for Categories

In [114]:
import json
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)

# Read and clean JSON files
def read_clean_json(file_path):
    data = []
    with open(file_path, 'r') as file:
        print(f'Reading from {file_path}')
        json_data = file.read()
        # print(f'After file.read:\n{json_data}')
        json_data = "[" + json_data.replace("}\n{", "},{") + "]"  # Format it as a list of JSON objects
        # print(f'After cleaning:\n{json_data}')
        data = json.loads(json_data)
        # print(f'After loading into list:\n{json_data}')
        
    return data

brands_data = read_clean_json('brands.json')
users_data = read_clean_json('users.json')
receipts_data = read_clean_json('receipts.json')

# Convert JSON data to DataFrames
brands_df = pd.DataFrame(brands_data)
users_df = pd.DataFrame(users_data)
receipts_df = pd.DataFrame(receipts_data)

Reading from brands.json
Reading from users.json
Reading from receipts.json


In [116]:
### Clean brands data
brands_df.info()
brands_df['_id'] = brands_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)
brands_df = brands_df.rename(columns={'_id': 'brand_id', 'brandCode': 'brand_code', 'categoryCode': 'category_code', 'topBrand': 'top_brand'})
brands_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           1167 non-null   object
 1   barcode       1167 non-null   object
 2   category      1012 non-null   object
 3   categoryCode  517 non-null    object
 4   cpg           1167 non-null   object
 5   name          1167 non-null   object
 6   topBrand      555 non-null    object
 7   brandCode     933 non-null    object
dtypes: object(8)
memory usage: 73.1+ KB


,brand_id,barcode,category,category_code,cpg,name,top_brand,brand_code
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, '$ref': 'Cogs'}",test brand @1612366101024,False,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, '$ref': 'Cogs'}",Starbucks,False,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, '$ref': 'Cogs'}",test brand @1612366146176,False,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, '$ref': 'Cogs'}",test brand @1612366146051,False,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, '$ref': 'Cogs'}",test brand @1612366146827,False,TEST BRANDCODE @1612366146827


In [118]:
# Function to extract the '$oid' and '$ref' from the nested dictionary
def extract_oid_ref(value):
    if isinstance(value, dict):
        oid = value.get('$id', {}).get('$oid', None)
        ref = value.get('$ref', None)
        # You can choose to return one or combine both
        return f"{oid} ({ref})" if ref else oid
    return value

brands_df['cpg'] = brands_df['cpg'].apply(extract_oid_ref)
brands_df.info()
brands_df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   brand_id       1167 non-null   object
 1   barcode        1167 non-null   object
 2   category       1012 non-null   object
 3   category_code  517 non-null    object
 4   cpg            1167 non-null   object
 5   name           1167 non-null   object
 6   top_brand      555 non-null    object
 7   brand_code     933 non-null    object
dtypes: object(8)
memory usage: 73.1+ KB


,brand_id,barcode,category,category_code,cpg,name,top_brand,brand_code
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550 (Cogs),test brand @1612366101024,False,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba (Cogs),Starbucks,False,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559 (Cogs),test brand @1612366146176,False,TEST BRANDCODE @1612366146176
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,601ac142be37ce2ead437559 (Cogs),test brand @1612366146051,False,TEST BRANDCODE @1612366146051
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7 (Cogs),test brand @1612366146827,False,TEST BRANDCODE @1612366146827
5,601ac142be37ce2ead43755b,511111719885,Baking,BAKING,601ac142be37ce2ead437559 (Cogs),test brand @1612366146091,False,TEST BRANDCODE @1612366146091
6,601ac142be37ce2ead43755c,511111219897,Baking,BAKING,601ac142be37ce2ead437559 (Cogs),test brand @1612366146133,False,TEST BRANDCODE @1612366146133
7,5cdad0f5166eb33eb7ce0faa,511111104810,Condiments & Sauces,NaN,559c2234e4b06aca36af13c6 (Cogs),J.L. Kraft,NaN,J.L. KRAFT
8,5ab15636e4b0be0a89bb0b07,511111504412,Canned Goods & Soups,NaN,5a734034e4b0d58f376be874 (Cogs),Campbell's Home Style,False,CAMPBELLS HOME STYLE
9,5c408e8bcd244a1fdb47aee7,511111504788,Baking,NaN,59ba6f1ce4b092b29c167346 (Cogs),test,NaN,TEST


In [265]:
# Step 1: Extract the required columns
categories_df = brands_df[['category_code', 'category']].copy()

# Step 2: Drop rows where 'category_code' is NaN since it is a primary key and it can't be null. 
categories_df = categories_df.dropna(subset=['category_code'])

# Step 3: Drop duplicates based on 'category_code'
categories_df = categories_df.drop_duplicates(subset=['category_code'])

# Step 4: Set 'category_code' as the index (or primary key)
categories_df = categories_df.set_index('category_code')

categories_df.head()

,category
category_code,
BEVERAGES,Beverages
BAKING,Baking
CANDY_AND_SWEETS,Candy & Sweets
CONDIMENTS_AND_SAUCES,Condiments & Sauces
CANNED_GOODS_AND_SOUPS,Canned Goods & Soups


In [121]:
users_df.info()
users_df['_id'] = users_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)
users_df['createdDate'] = users_df['createdDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
users_df['lastLogin'] = users_df['lastLogin'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
users_df = users_df.rename(columns={'_id': 'user_id', 'createdDate': 'created_date', 'lastLogin': 'last_login', 'signUpSource': 'sign_up_source'})
users_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


,user_id,active,created_date,last_login,role,sign_up_source,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI


In [123]:
receipts_df.info()
receipts_df['_id'] = receipts_df['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)
receipts_df['createDate'] = receipts_df['createDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df['dateScanned'] = receipts_df['dateScanned'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df['finishedDate'] = receipts_df['finishedDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df['modifyDate'] = receipts_df['modifyDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df['pointsAwardedDate'] = receipts_df['pointsAwardedDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df['purchaseDate'] = receipts_df['purchaseDate'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else x)
receipts_df = receipts_df.rename(columns={'_id': 'receipt_id', 'bonusPointsEarned': 'bonus_points_earned', 'bonusPointsEarnedReason': 'bonus_points_earned_reason', 'createDate': 'create_date', 'dateScanned': 'date_scanned', 'finishedDate': 'finished_date', 'modifyDate': 'modify_date', 'pointsAwardedDate': 'points_awarded_date', 'pointsEarned': 'points_earned', 'purchaseDate': 'purchase_date', 'purchasedItemCount': 'purchase_item_count', 'rewardsReceiptItemList': 'rewards_receipt_item_list', 'rewardsReceiptStatus': 'rewards_receipt_status', 'totalSpent': 'total_spent', 'userId': 'user_id'})
receipts_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    object 
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    object 
 14  userId                  

,receipt_id,bonus_points_earned,bonus_points_earned_reason,create_date,date_scanned,finished_date,modify_date,points_awarded_date,points_earned,purchase_date,purchase_item_count,rewards_receipt_item_list,rewards_receipt_status,total_spent,user_id
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '26.00', 'itemPrice': '26.00...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '1', 'itemPrice': '1', 'part...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5,1.609632e+12,1.0,"[{'needsFetchReview': False, 'partnerItemId': '1', 'preventTargetGapPoints': True, 'userFlaggedB...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '28.00', 'itemPrice': '28.00...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '1', 'itemPrice': '1', 'part...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [125]:
# Explode the rewards_receipt_item_list so each item becomes a row
df_exploded = receipts_df.explode('rewards_receipt_item_list')

# Normalize the rewards_receipt_item_list (which is a JSON object) into its own columns
items_df = pd.json_normalize(df_exploded['rewards_receipt_item_list'])

# Resetting indices
df_exploded.reset_index(drop=True, inplace=True)
items_df.reset_index(drop=True, inplace=True)

# Adding a receipt_item_id
# Create a unique receipt_item_id for each item by combining receipt_id and a counter
df_exploded['receipt_item_id'] = df_exploded.groupby('receipt_id').cumcount() + 1
df_exploded['receipt_item_id'] = df_exploded['receipt_id'] + '_item_' + df_exploded['receipt_item_id'].astype(str)

# Concatenating DataFrames
receipt_items_df = pd.concat([df_exploded['receipt_id'], items_df, df_exploded['receipt_item_id']], axis=1)

# Rename column names
receipt_items_df = receipt_items_df.rename(columns={'finalPrice': 'final_price', 'itemPrice': 'item_price', 'needsFetchReview': 'needs_fetch_review', 'partnerItemId': 'partner_item_id', 'preventTargetGapPoints': 'prevent_target_gap_points', 'quantityPurchased': 'quantity_purchased', 'userFlaggedBarcode': 'user_flagged_barcode', 'userFlaggedNewItem': 'user_flagged_new_item', 'userFlaggedPrice': 'user_flagged_price', 'userFlaggedQuantity': 'user_flagged_quantity', 'needsFetchReviewReason': 'needs_fetch_review_reason', 'pointsNotAwardedReason': 'points_not_awarded_reason', 'pointsPayerId': 'points_payer_id', 'rewardsGroup': 'rewards_group', 'rewardsProductPartnerId': 'rewards_product_partner_id', 'userFlaggedDescription': 'user_flagged_description', 'originalMetaBriteBarcode': 'original_meta_brite_barcode', 'originalMetaBriteDescription': 'original_meta_brite_description', 'brandCode': 'brand_code', 'competitorRewardsGroup': 'competitor_rewards_group', 'discountedItemPrice': 'discounted_item_price', 'originalReceiptItemText': 'original_receipt_item_text', 'itemNumber': 'item_number', 'originalMetaBriteQuantityPurchased': 'original_meta_brite_quantity_purchased', 'pointsEarned': 'points_earned', 'targetPrice': 'target_price', 'competitiveProduct': 'competitive_product', 'originalFinalPrice': 'original_final_price', 'originalMetaBriteItemPrice': 'original_meta_brite_item_price', 'priceAfterCoupon': 'price_after_coupon', 'metabriteCampaignId': 'metabrite_campaign_id'})

receipt_items_df.head()

,receipt_id,barcode,description,final_price,item_price,needs_fetch_review,partner_item_id,prevent_target_gap_points,quantity_purchased,user_flagged_barcode,user_flagged_new_item,user_flagged_price,user_flagged_quantity,needs_fetch_review_reason,points_not_awarded_reason,points_payer_id,rewards_group,rewards_product_partner_id,user_flagged_description,original_meta_brite_barcode,original_meta_brite_description,brand_code,competitor_rewards_group,discounted_item_price,original_receipt_item_text,item_number,original_meta_brite_quantity_purchased,points_earned,target_price,competitive_product,original_final_price,original_meta_brite_item_price,deleted,price_after_coupon,metabrite_campaign_id,receipt_item_id
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1eb0a720f0523000575_item_1
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1bb0a720f052300056b_item_1
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ,10.00,10.00,True,2,True,1.0,028400642255,True,10.00,1.0,USER_FLAGGED,Action not allowed for user and CPG,5332f5fbe4b03c9a25efd0ba,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1bb0a720f052300056b_item_2
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,26.00,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1f10a720f052300057a_item_1
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5ff1e1ee0a7214ada100056f_item_1


In [127]:
receipt_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 36 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   receipt_id                              7381 non-null   object 
 1   barcode                                 3090 non-null   object 
 2   description                             6560 non-null   object 
 3   final_price                             6767 non-null   object 
 4   item_price                              6767 non-null   object 
 5   needs_fetch_review                      813 non-null    object 
 6   partner_item_id                         6941 non-null   object 
 7   prevent_target_gap_points               358 non-null    object 
 8   quantity_purchased                      6767 non-null   float64
 9   user_flagged_barcode                    337 non-null    object 
 10  user_flagged_new_item                   323 non-null    obje

### Explore Data Quality Issues

#### 1. Completeness: Check for missing values (Handling: Remove missing values or impute values using other values)

In [131]:
# Check for missing values
def check_missing_values(df, name):
    missing_values = df.isnull().sum()
    total_records = df.shape[0]
    missing_percentage = ((missing_values / total_records) * 100).round(0)
    
    # Create a DataFrame for better output formatting
    missing_df = pd.DataFrame({
        'Missing Values': missing_values,
        '% of Total Records': missing_percentage
    })
    
    print(f"Missing values in {name} ({total_records} records):\n{missing_df}\n")

check_missing_values(brands_df, 'Brands')
check_missing_values(users_df, 'Users')
check_missing_values(receipts_df, 'Receipts')
check_missing_values(receipt_items_df, 'Receipt Items')

Missing values in Brands (1167 records):
               Missing Values  % of Total Records
brand_id                    0                 0.0
barcode                     0                 0.0
category                  155                13.0
category_code             650                56.0
cpg                         0                 0.0
name                        0                 0.0
top_brand                 612                52.0
brand_code                234                20.0

Missing values in Users (495 records):
                Missing Values  % of Total Records
user_id                      0                 0.0
active                       0                 0.0
created_date                 0                 0.0
last_login                  62                13.0
role                         0                 0.0
sign_up_source              48                10.0
state                       56                11.0

Missing values in Receipts (1119 records):
                 

In [133]:
# Handle missing values

# Brands 
# Keep brand_code as missing (NaN)
# Replace NaN values in 'top_brand' column with False
# Suppress future warning and ensure proper downcasting
brands_df['top_brand'] = brands_df['top_brand'].fillna(False)
brands_df['top_brand'] = brands_df['top_brand'].infer_objects(copy=False)

def clean_category_data(row):
    # Case 1: Both category and category_code are NaN, remove the row
    if pd.isna(row['category']) and pd.isna(row['category_code']):
        return None  # Indicate that the row should be removed

    # Case 2: category is NaN, clean and assign category_code to category
    if pd.isna(row['category']):
        row['category'] = row['category_code'].lower().replace('_', ' ').replace('and', '&').title()  # Mixed case (title case)
        
    # Case 3: category_code is NaN, clean and assign category to category_code
    if pd.isna(row['category_code']):
        row['category_code'] = row['category'].upper().replace(' ', '_').replace('&', 'AND')

    return row

# Apply the clean_category_data function to each row
brands_df = brands_df.apply(clean_category_data, axis=1)
# Drop any rows where the function returned None (rows where both were NaN)
brands_df = brands_df.dropna(how='any')

# Users
# Keep state & sign_up_source as missing (NaN)
# Impute missing last_login with NaT
users_df['last_login'] = pd.to_datetime(users_df['last_login'], errors='coerce')  # Convert to datetime
users_df['last_login'] = users_df['last_login'].fillna(pd.NaT) 

# Receipts WIP
# Receipt Items WIP  

check_missing_values(brands_df, 'Brands')
check_missing_values(users_df, 'Users')
# check_missing_values(receipts_df, 'Receipts')
# check_missing_values(receipt_items_df, 'Receipt Items')

Missing values in Brands (929 records):
               Missing Values  % of Total Records
brand_id                    0                 0.0
barcode                     0                 0.0
category                    0                 0.0
category_code               0                 0.0
cpg                         0                 0.0
name                        0                 0.0
top_brand                   0                 0.0
brand_code                  0                 0.0

Missing values in Users (495 records):
                Missing Values  % of Total Records
user_id                      0                 0.0
active                       0                 0.0
created_date                 0                 0.0
last_login                  62                13.0
role                         0                 0.0
sign_up_source              48                10.0
state                       56                11.0



/var/folders/f0/fr1ltxqj5fn40243wq6j_d300000gp/T/ipykernel_6044/4004765895.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  brands_df['top_brand'] = brands_df['top_brand'].fillna(False)


#### 2. Uniqueness: Check for duplicate entries (Handling: remove duplicates)

In [135]:
def check_duplicates(df, name, id_column):
    duplicates = df[df.duplicated(subset=[id_column])]
    num_duplicates = duplicates.shape[0]
    total_records = df.shape[0]
    duplicates_percentage = round((num_duplicates / total_records) * 100, 0)
    
    if num_duplicates: 
        print(f"{name} Table: {num_duplicates} duplicates found ({duplicates_percentage}% out of {total_records} total records).\n{duplicates}\n")
        # Optional: Handle duplicates
        handle_duplicates(df, name, id_column)
    else: 
        print(f"{name} Table: No duplicates found.")

def handle_duplicates(df, name, id_column):
    # Remove duplicates and keep the first occurrence
    df = df.drop_duplicates(subset=[id_column], keep='first')
    print(f"Missing values removed from {name} table. Current record count: {df.shape[0]}.")
    
check_duplicates(brands_df, 'Brands', 'brand_id')
check_duplicates(users_df, 'Users', 'user_id')
check_duplicates(receipts_df, 'Receipts', 'receipt_id')
check_duplicates(receipt_items_df, 'Receipt Items', 'receipt_item_id')

Brands Table: No duplicates found.
Users Table: 283 duplicates found (57.0% out of 495 total records).
                      user_id  active   created_date  \
1    5ff1e194b6a9d73a3a9f1052    True  1609687444800   
2    5ff1e194b6a9d73a3a9f1052    True  1609687444800   
4    5ff1e194b6a9d73a3a9f1052    True  1609687444800   
5    5ff1e194b6a9d73a3a9f1052    True  1609687444800   
8    5ff1e194b6a9d73a3a9f1052    True  1609687444800   
..                        ...     ...            ...   
490  54943462e4b07e684157a532    True  1418998882381   
491  54943462e4b07e684157a532    True  1418998882381   
492  54943462e4b07e684157a532    True  1418998882381   
493  54943462e4b07e684157a532    True  1418998882381   
494  54943462e4b07e684157a532    True  1418998882381   

                       last_login         role sign_up_source state  
1   1970-01-01 00:26:49.687537858     consumer          Email    WI  
2   1970-01-01 00:26:49.687537858     consumer          Email    WI  
4   1970-01-01

#### 3. Conformity: Check for invalid data types (Handling: Cast into valid data types)

In [143]:
# 3. Check for invalid data types
def check_data_types(df, name):
    print(f"Data types in {name} table:\n{df.dtypes}\n")

check_data_types(brands_df, 'Brands')
check_data_types(users_df, 'Users')
check_data_types(receipts_df, 'Receipts')
check_data_types(receipt_items_df, 'Receipt Items')

Data types in Brands table:
brand_id         object
barcode          object
category         object
category_code    object
cpg              object
name             object
top_brand        object
brand_code       object
dtype: object

Data types in Users table:
user_id                   object
active                      bool
created_date               int64
last_login        datetime64[ns]
role                      object
sign_up_source            object
state                     object
dtype: object

Data types in Receipts table:
receipt_id                     object
bonus_points_earned           float64
bonus_points_earned_reason     object
create_date                     int64
date_scanned                    int64
finished_date                 float64
modify_date                     int64
points_awarded_date           float64
points_earned                  object
purchase_date                 float64
purchase_item_count           float64
rewards_receipt_item_list      object
reward

In [214]:
brands_df['top_brand'] = brands_df['top_brand'].astype(bool)
brands_df['brand_id'] = brands_df['brand_id'].astype('string')
brands_df['barcode'] = brands_df['barcode'].astype('string')
brands_df['category'] = brands_df['category'].astype('string')
brands_df['category_code'] = brands_df['category_code'].astype('string')
brands_df['cpg'] = brands_df['cpg'].astype('string')
brands_df['name'] = brands_df['name'].astype('string')
brands_df['brand_code'] = brands_df['brand_code'].astype('string')

users_df['user_id'] = users_df['user_id'].astype('string')
users_df['created_date'] = pd.to_datetime(users_df['created_date'], unit='s', errors='coerce')  # Convert Unix timestamp to datetime
users_df['last_login'] = pd.to_datetime(users_df['last_login'], unit='s', errors='coerce')  # Convert and handle errors
users_df['role'] = users_df['role'].astype('string')
users_df['sign_up_source'] = users_df['sign_up_source'].astype('string')
users_df['state'] = users_df['state'].astype('string')

receipts_df['user_id'] = receipts_df['user_id'].astype('string')
receipts_df['receipt_id'] = receipts_df['receipt_id'].astype('string')
receipts_df['create_date'] = pd.to_datetime(receipts_df['create_date'], unit='s', errors='coerce')
receipts_df['date_scanned'] = pd.to_datetime(receipts_df['date_scanned'], unit='s', errors='coerce')
receipts_df['finished_date'] = pd.to_datetime(receipts_df['finished_date'], unit='s', errors='coerce')
receipts_df['modify_date'] = pd.to_datetime(receipts_df['modify_date'], unit='s', errors='coerce')
receipts_df['points_awarded_date'] = pd.to_datetime(receipts_df['points_awarded_date'], unit='s', errors='coerce')
receipts_df['points_earned'] = pd.to_numeric(receipts_df['points_earned'], errors='coerce')
receipts_df['purchase_date'] = pd.to_datetime(receipts_df['purchase_date'], unit='s', errors='coerce')
receipts_df['total_spent'] = pd.to_numeric(receipts_df['total_spent'], errors='coerce')

receipt_items_df['receipt_item_id'] = receipt_items_df['receipt_item_id'].astype('string')
receipt_items_df['receipt_id'] = receipt_items_df['receipt_id'].astype('string')
receipt_items_df['final_price'] = pd.to_numeric(receipt_items_df['final_price'], errors='coerce')
receipt_items_df['item_price'] = pd.to_numeric(receipt_items_df['item_price'], errors='coerce')
receipt_items_df['quantity_purchased'] = receipt_items_df['quantity_purchased'].astype(float)  # ensure float

check_data_types(brands_df, 'Brands')
check_data_types(users_df, 'Users')
check_data_types(receipts_df, 'Receipts')
check_data_types(receipt_items_df, 'Receipt Items')

Data types in Brands table:
brand_id         string[python]
barcode          string[python]
category         string[python]
category_code    string[python]
cpg              string[python]
name             string[python]
top_brand                  bool
brand_code       string[python]
dtype: object

Data types in Users table:
user_id           string[python]
active                      bool
created_date      datetime64[ns]
last_login        datetime64[ns]
role              string[python]
sign_up_source    string[python]
state             string[python]
dtype: object

Data types in Receipts table:
receipt_id                    string[python]
bonus_points_earned                  float64
bonus_points_earned_reason            object
create_date                   datetime64[ns]
date_scanned                  datetime64[ns]
finished_date                 datetime64[ns]
modify_date                   datetime64[ns]
points_awarded_date           datetime64[ns]
points_earned                        f

#### 4. Consistency/Accuracy: Check for inconsistencies or outliers in numerical columns

In [224]:
# 4. Check for inconsistencies or outliers in numerical columns
def check_numerical_outliers(df, name):
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    for col in numerical_cols:
        print(f"Outliers in {name} - {col}:\n{df[col].describe()}\n")

# check_numerical_outliers(brands_df, 'Brands') # no numerical columns
# check_numerical_outliers(users_df, 'Users') # no numerical columns
check_numerical_outliers(receipts_df, 'Receipts')
check_numerical_outliers(receipt_items_df, 'Receipt Items')


Outliers in Receipts - bonus_points_earned:
count    544.000000
mean     238.893382
std      299.091731
min        5.000000
25%        5.000000
50%       45.000000
75%      500.000000
max      750.000000
Name: bonus_points_earned, dtype: float64

Outliers in Receipts - points_earned:
count      609.000000
mean       585.962890
std       1357.166947
min          0.000000
25%          5.000000
50%        150.000000
75%        750.000000
max      10199.800000
Name: points_earned, dtype: float64

Outliers in Receipts - purchase_item_count:
count    635.00000
mean      14.75748
std       61.13424
min        0.00000
25%        1.00000
50%        2.00000
75%        5.00000
max      689.00000
Name: purchase_item_count, dtype: float64

Outliers in Receipts - total_spent:
count     684.000000
mean       77.796857
std       347.110349
min         0.000000
25%         1.000000
50%        18.200000
75%        34.960000
max      4721.950000
Name: total_spent, dtype: float64

Outliers in Receipt Item

#### 5. Consistency/Accuracy: Check for logical inconsistencies (e.g., future dates, negative values)

In [246]:
# 5. Check for logical inconsistencies (e.g., future dates, negative values)
def check_logical_inconsistencies(df, name):
    if 'created_date' in df.columns:
        future_dates = df[df['created_date'] > pd.Timestamp.now()]
        print(f"Future dates in {name}:\n{future_dates}\n")
        
    if 'last_login' in df.columns: 
        future_dates = df[df['last_login'] > pd.Timestamp.now()]
        print(f"Future dates in {name}:\n{future_dates}\n")
        
    if 'create_date' in df.columns:
        future_dates = df[df['create_date'] > pd.Timestamp.now()]
        print(f"Future dates in {name}:\n{future_dates}\n")
        
    if 'total_spent' in df.columns:
        negative_spent = df[df['total_spent'] < 0]
        print(f"Negative total spent in {name}:\n{negative_spent}\n")

# check_logical_inconsistencies(brands_df, 'Brands') # no date columns 
check_logical_inconsistencies(users_df, 'Users')
check_logical_inconsistencies(receipts_df, 'Receipts')
# check_logical_inconsistencies(receipt_items_df, 'Receipt Items') # no date columns 

Future dates in Users:
Empty DataFrame
Columns: [user_id, active, created_date, last_login, role, sign_up_source, state]
Index: []

Future dates in Users:
Empty DataFrame
Columns: [user_id, active, created_date, last_login, role, sign_up_source, state]
Index: []

Future dates in Receipts:
Empty DataFrame
Columns: [receipt_id, bonus_points_earned, bonus_points_earned_reason, create_date, date_scanned, finished_date, modify_date, points_awarded_date, points_earned, purchase_date, purchase_item_count, rewards_receipt_item_list, rewards_receipt_status, total_spent, user_id]
Index: []

Negative total spent in Receipts:
Empty DataFrame
Columns: [receipt_id, bonus_points_earned, bonus_points_earned_reason, create_date, date_scanned, finished_date, modify_date, points_awarded_date, points_earned, purchase_date, purchase_item_count, rewards_receipt_item_list, rewards_receipt_status, total_spent, user_id]
Index: []



#### 6. Integrity: Check for relationships between tables (e.g., foreign key constraints)

In [252]:
# Check for duplicates in users DataFrame
duplicate_users = users_df[users_df.duplicated('user_id', keep=False)]
if not duplicate_users.empty:
    print("Duplicate user IDs found:")
    print(duplicate_users)
else:
    print("No duplicate user IDs found.")


# Find user IDs in receipts_df that are not in users_df
invalid_user_ids = receipts_df[~receipts_df['user_id'].isin(users_df['user_id'])]

if not invalid_user_ids.empty:
    print("Invalid user IDs found in receipts_df:")
    print(invalid_user_ids[['user_id']])
else:
    print("All user IDs in receipts_df are valid.")

# Count receipts per user
receipts_count = receipts_df['user_id'].value_counts()
print("Number of receipts per user:")
print(receipts_count)


Duplicate user IDs found:
                      user_id  active created_date  \
0    5ff1e194b6a9d73a3a9f1052    True          NaT   
1    5ff1e194b6a9d73a3a9f1052    True          NaT   
2    5ff1e194b6a9d73a3a9f1052    True          NaT   
3    5ff1e1eacfcf6c399c274ae6    True          NaT   
4    5ff1e194b6a9d73a3a9f1052    True          NaT   
..                        ...     ...          ...   
490  54943462e4b07e684157a532    True          NaT   
491  54943462e4b07e684157a532    True          NaT   
492  54943462e4b07e684157a532    True          NaT   
493  54943462e4b07e684157a532    True          NaT   
494  54943462e4b07e684157a532    True          NaT   

                       last_login         role sign_up_source state  
0   1970-01-01 00:26:49.687537858     consumer          Email    WI  
1   1970-01-01 00:26:49.687537858     consumer          Email    WI  
2   1970-01-01 00:26:49.687537858     consumer          Email    WI  
3   1970-01-01 00:26:49.687530597     consume

In [254]:
# Check for duplicates in receipts DataFrame
duplicate_receipts = receipts_df[receipts_df.duplicated('receipt_id', keep=False)]
if not duplicate_receipts.empty:
    print("Duplicate receipt IDs found:")
    print(duplicate_receipts)
else:
    print("No duplicate receipt IDs found.")

# Find receipt IDs in receipt_items_df that are not in receipts_df
invalid_receipt_ids = receipt_items_df[~receipt_items_df['receipt_id'].isin(receipts_df['receipt_id'])]

if not invalid_receipt_ids.empty:
    print("Invalid receipt IDs found in receipt_items_df:")
    print(invalid_receipt_ids[['receipt_id']])
else:
    print("All receipt IDs in receipt_items_df are valid.")

# Count receipt items per receipt
items_count = receipt_items_df['receipt_id'].value_counts()
print("Number of items per receipt:")
print(items_count)


No duplicate receipt IDs found.
All receipt IDs in receipt_items_df are valid.
Number of items per receipt:
receipt_id
600f2fc80a720f0535000030    459
600f39c30a7214ada2000030    450
600f24970a720f053500002f    381
600f0cc70a720f053500002c    217
600a1a8d0a7214ada2000008    203
                           ... 
6024b5b70a720f05a80001d6      1
603a16620a7217c72c0001b8      1
603abdf60a7217c72c00024d      1
60267a190a7214d8e90002fa      1
603c4fea0a7217c72c000389      1
Name: count, Length: 1119, dtype: Int64


# Fourth: Communicate with Stakeholders

#### Sample Email to Product Leader
#### Subject: Insights on Data Quality and Potential Improvements for Upcoming Product Reports

Hi Product Leader,

I’ve been reviewing the data provided for our upcoming analysis, and I wanted to give you a quick update on a few data quality issues I've discovered. Identifying these early helps ensure we get the most accurate results for the analysis. Here’s a high-level overview of the Data Quality checks performed and some findings:
1. Missing Data (Data Completeness): Several records across the Users, Brands, Recripts datasets have missing values, such as User's last login date, state and sign up source. This could lead to gaps in our reports if left unresolved. In Brands, more than 50% of records have missing values for category code and top brand, which can be imputed with values from category and false respectively in order to complete the dataset. In Receipts, more than 50% of records have missing values for bonus points earned, bonus points earned reason and points awarded date, which can be deleted or imputed in with some default values. In Receipt Items, many fields are present with more than 50% of records having missing values, hence these should be corrected to give a more accurate results from the analysis.
2. Duplicate Entries (Data Uniqueness): 57% of the records in the Users table contain duplicate records, which inflates the total number of users in our analysis. These can be deleted to not impact the analysis.
3. Invalid Data Types (Data Conformity): Some of the data types are read in as Objects format, hence we have modified it to one of either string, boolean, numeric or datetime fields.
4. Inconsistent Data Formatting (Data Consistency/Accuracy): We checked for any inconsistencies or outliers in the numerical fields and any logical inconsistencies in other fields (like future dates or negative values), but didn't find any major issues. Some extreme values in points earned and total spent seem to be skewing our metrics. These might need closer investigation to determine if they’re valid or should be excluded from certain analyses.
5. Invalid relationships between tables (Data Integrity): There are some user ids in Receipts table which have no corresponding record in the Users table, hence the foreign key constraint is not met. This could skew the insights we gather from the data, hence should be removed or corrected.

To address these issues, I would recommend:
1. Data Completeness --> Confirm the correct values for missing fields or delete those records.
2. Data Uniqueness --> Clarify whether duplicate entries are valid (e.g., repeat scans or system errors), otherwise delete records.
3. Data Conformity --> Verify the data types from the relevant teams owning the data.
4. Data Consistency/Accuracy --> Clarify whether to remove outliers or they are accurate as they are.
5. Data Integrity --> Clarify if these records should be corrected or removed. 

Other questions about the data:  
1. Should missing records in Users, Brands or Receipts table be removed or imputed? For example, category and category code seem to be similar to each other and can be programmatically imputed where possible. Top brand can be filled with false as a default value. Similarly we could design default values for other fields as well. 
2. Should duplicate entries in Users be kept as valid users, or should we de-duplicate these for reporting?
3. Receipt Items table contains no brand_id, does this mean there is no way to relate items purchased on a receipt to the brands they belong to? Since there is barcode and brand_code, could we add brand_id? This would help to produce meaningful insights for some of the predetermined questions that you have. (refer to Second question in the exercise)
4. How was the receipt data  captured (manual entry vs automated system) to understand why some contain extreme values.
5. Insight into whether the business logic allows for multiple scans of the same receipt, as this affects how we handle duplicate entries or entries in Receipt table which have been linked to user ids that do not exist in the Users table.
6. Understanding of the brand and merchant relationships to ensure our reporting aligns with the business goals for marketing or sales campaigns.

I have designed my data model in order for query flexibility, efficiency and normalization. As the systems get larger, normalization reduces data redundancy by separating Users, Receipts, Brands, Receipt Items and Cateogories into distinct entities. It is also easier to maintin and update data without duplication. The relational structure also enables complex queries across the tables. However, as we think about productionizing this system, here are some Performance/Scaling concerns:
1. Queries involving large tables (e.g., receipts and receipt items) could slow down if not indexed appropriately. We would need to ensure that the Receipts table is properly indexed, particularly on key columns such as receipt_id and the Receipt Items table on receipt_item_id.
2. As transaction data grows, we might face challenges in managing joins across large datasets (e.g., joining Users and Receipts tables). Using partitioning techniques (e.g., by month on the create_date or purchase_date) could help maintain performance as the volume increases.
3. Aggregating data over long periods could also strain the system, so implementing materialized views or pre-aggregating common metrics might help optimize the analytics pipeline.
To address these concerns, I’d propose running stress tests on our queries and adjusting indexing and partitioning strategies accordingly.

Could we also set up a quick call to understand how this data was ingested/processed initially? This will help us ensure that future data quality standards are met by establishing a data validation process going forward to catch these errors earlier.

Let me know if you have any questions or need additional details!

Best,

Ipsita Mohapatra

Analytics Engineer

